In [1]:
import const
import json
import pandas as pd
import numpy

In [2]:
squad_ds_path: str = const.DATASETS_FOLDER + "train-v2.0.json"

In [3]:
squad_ds_json: dict = {}

In [4]:
with open(squad_ds_path, 'r') as f:
    squad_ds_json = json.load(f)

In [5]:
squad_ds_json.keys()

dict_keys(['version', 'data'])

In [6]:
def process_squad_sub_dict(data_dict, present_path, subpath_list, subpath_ls_ind=None, topic_nm = None, pg=None):
    """Processes the sub dictionaries from the SQUAD dataset and returns questions
        as pandas dataframe.

    PARAMETERS
    present_path - The current path in the data_dict
    subpath_list - The list of keys in the current path
    subpath_ls_ind - The current index to process the subpath.
    topic_nm - The name of the topic which this function instance is processing.
    """

    if subpath_ls_ind is None:
        path_depth = len(present_path.split("/"))
        if path_depth < 6:
            subpath_ls_ind = len(subpath_list) - 1
            squad_df = process_squad_sub_dict(data_dict, present_path, subpath_list, subpath_ls_ind, topic_nm, pg)
            return squad_df
        else:
            if path_depth == 6:
                if (topic_nm is not None) & (pg is not None):
                    try:
                        new_subpath_ls = list(data_dict.keys())
                    except:
                        if isinstance(data_dict, keys):
                            new_subpath_len = len(data_dict)
                            new_subpath_ls = []
                            for i in range(new_subpath_len):
                                new_subpath_ls.append(i)
                    lowest_path = present_path.split("/")
                    path_depth = len(lowest_path) 
                    lowest_path = lowest_path[path_depth-2]
                    if lowest_path == "context":
                        return data_dict
                    else:
                        is_impossible = data_dict['is_impossible']

                        ##TODO: HANDLE CONCRETE ANSWERS AND PLAUSSIBLE ANSWERS.
                        answers = (process_squad_sub_dict(data_dict, present_path, new_subpath_ls, subpath_ls_ind=None, 
                              topic_nm=topic_nm, pg=pg))
                        sample_df: pd.DataFrame = (pd.DataFrame(data={'topic': topic_nm, 'paragraph': pg, 'question_content': 
                             data_dict['question'],'question_id': data_dict['id'], 'answers': answers}))
            else:
                if path_depth == 7:
                    return data_dict
    else:
        path_depth = len(present_path.split("/"))
        if path_depth < 8:
            new_path = present_path + str(subpath_list[subpath_ls_ind]) + "/"
            if isinstance(subpath_list[0], str):
                sub_dict = data_dict[subpath_list[subpath_ls_ind]]
            else:
                sub_dict = data_dict[subpath_ls_ind]
                
            try:
                new_subpath_ls = list(sub_dict.keys())
            except:
                new_subpath_len = len(sub_dict)
                new_subpath_ls = []
                for i in range(new_subpath_len):
                    new_subpath_ls.append(i)
                
                    
            topic = topic_nm
            new_pg = pg
            if path_depth == 3:
                topic = data_dict['title']
            elif (path_depth == 4) or (path_depth == 6):
                if (isinstance(data_dict, list) == False) and (isinstance(data_dict, dict) == False):
                    return data_dict
                else:
                    pass
            elif path_depth == 5:
                new_pg = data_dict['context']
            elif path_depth == 8:
                print(data_dict)

            ## TODO: SET BREAKPOINTS HERE FOR DEBUGGING.
            if subpath_ls_ind > 0:
                temp = process_squad_sub_dict(sub_dict, present_path, subpath_list, topic_nm=topic, pg=new_pg, subpath_ls_ind=subpath_ls_ind-1)
            else:
                return process_squad_sub_dict(sub_dict, new_path, new_subpath_ls, topic_nm=topic, pg=new_pg)
            
            
    

In [7]:
def extract_questions_only(json_dict: dict, present_path=None):
    """
        This recursive function extracts the questions from the SQUAD
        dataset. It returns the questions as a Pandas dataframe.
        PARAMETERS
        json_dict - The dictionary to process this function.
        path - The path from the root level in the json dictionary
            that is being processed.
    """

    if present_path is None:
        sub_dict = json_dict['data']
        new_path = 'data/'
        squad_df = extract_questions_only(sub_dict, new_path)
    elif present_path == 'data/':
        subpath_len = len(json_dict)
        sub_paths = []
        for i in range(subpath_len):
            sub_paths.append(i)
        return process_squad_sub_dict(json_dict, present_path, sub_paths)

    return squad_df
        

In [8]:
squad_df = extract_questions_only(squad_ds_json)